In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import os
from os import listdir
from os.path import isfile, join

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.ecephys_session import (
    EcephysSession, 
    removed_unused_stimulus_presentation_columns
)
from allensdk.brain_observatory.ecephys.visualization import plot_mean_waveforms, plot_spike_counts, raster_plot

# tell pandas to show all columns when we display a DataFrame
pd.set_option("display.max_columns", None)

In [2]:
layer_borders = [0, 100, 310, 430, 650, 850] # [1E-6*m]
layerpop_names = ['e23', 'e4', 'e5', 'e6', 'i1', 'i23', 'i4', 'i5', 'i6']
nlayers = 5

In [3]:
plt.rcParams.update({'font.size': 16})

## Load data

In [4]:
exp_sessions_data_dir = '/Volumes/My Passport/ecephys_cache_dir_10_31'

manifest_path = os.path.join(exp_sessions_data_dir, "manifest.json")

cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

In [6]:
list_session_ids = [int(f.path.split('/')[-1].split('_')[-1]) for f in os.scandir(exp_sessions_data_dir ) if f.is_dir()]
list_session_ids_string = [f.path.split('/')[-1].split('_')[-1] for f in os.scandir(exp_sessions_data_dir ) if f.is_dir()]
nsessions = len(list_session_ids)

In [7]:
list_session_ids = [list_session_ids[0]]
list_session_ids

[715093703]

In [8]:
list_sessions = [cache.get_session_data(session_id) for session_id in list_session_ids]

In [9]:
channels = cache.get_channels()
units = cache.get_units()

#### Find depth of units in each session

In [10]:
units_VISp_session_depth = []
for i, session_id in enumerate(list_session_ids):
    # Get channels in VISp for each session
    channels_session = channels.loc[channels['ecephys_session_id'] == session_id]
    channels_VISp_session = channels_session.loc[channels_session['ecephys_structure_acronym'] == 'VISp']
    
    # Get units in VISp for each session
    units_session = units.loc[units['ecephys_session_id'] == session_id]
    units_VISp_session = units_session.loc[units_session['ecephys_structure_acronym'] == 'VISp']
    
    # Distance from tip of probe is depth of each unit
    units_VISp_session_depth_temp = abs(units_VISp_session['probe_vertical_position']-channels_VISp_session['probe_vertical_position'].max())
    units_VISp_session_depth.append(units_VISp_session_depth_temp)

### Identifying which units are excitatory and inhibitory based on peak-to-peak width
- Smaller p2p width than 0.4 ms --> fast-spiking, i.e. inhibitory cells
- Greater p2p width than 0.4 ms --> regular-spiking, i.e. excitatory cells
- Otherwise: Cannot determine whether cell is inhibitory of excitatory based on waveform, so it is disregarded
- Note: This calculation takes some time

In [11]:
exc_nrns_list = []; inh_nrns_list = []

for i, session in enumerate(list_sessions):
    units_of_interest = units_VISp_session_depth[i].index.to_numpy()
    
    waveforms = {uid: session.mean_waveforms[uid] for uid in units_of_interest}
    peak_channels = {uid: session.units.loc[uid, 'peak_channel_id'] for uid in units_of_interest}
    
    p2p_width = np.zeros(len(units_of_interest))
    
    
    nrn_type_mark = {} # Did not end up using this
    
    exc_nrns_list_temp = []; inh_nrns_list_temp = []
    p2p_cut_off = 0.4 # (ms)
    for i, unit_nr in enumerate(units_of_interest):
        mean_waveform = np.mean(waveforms[unit_nr], axis = 0)
        time_max_peak = waveforms[unit_nr].time[np.argmax(mean_waveform)]
        #print('time_max_peak: ', time_max_peak)
        time_min_peak = waveforms[unit_nr].time[np.argmin(mean_waveform)]
        #print('time_min_peak: ', time_min_peak)
        p2p_width[i] = (time_max_peak-time_min_peak)*1E3
        print('p2p_width: ', p2p_width[i])

        if p2p_width[i] > p2p_cut_off:
            nrn_type_mark[unit_nr] = 'e' # excitatory
            exc_nrns_list_temp.append(unit_nr)
        elif p2p_width[i] < p2p_cut_off and p2p_width[i] > 0:
            nrn_type_mark[unit_nr] = 'i' # inhibitory
            inh_nrns_list_temp.append(unit_nr)
        else:
            print("Can't determine whether inhibitory or excitatory. Disregard.")

    exc_nrns_list.append(np.asarray(exc_nrns_list_temp))
    inh_nrns_list.append(np.asarray(inh_nrns_list_temp))

p2p_width:  0.5000002421731657
p2p_width:  -0.6666669895642209
Can't determine whether inhibitory or excitatory. Disregard.
p2p_width:  1.0333338338245424
p2p_width:  0.3666668442603216
p2p_width:  0.7333336885206431
p2p_width:  1.9000009202580297
p2p_width:  0.7666670379988542
p2p_width:  -0.16666674739105528
Can't determine whether inhibitory or excitatory. Disregard.
p2p_width:  -0.766667037998854
Can't determine whether inhibitory or excitatory. Disregard.
p2p_width:  0.43333354321674356
p2p_width:  0.3666668442603216
p2p_width:  0.700000339042432
p2p_width:  0.733333688520643
p2p_width:  2.0000009686926625
p2p_width:  -0.5000002421731656
Can't determine whether inhibitory or excitatory. Disregard.
p2p_width:  0.6000002906077988
p2p_width:  0.6333336400860099
p2p_width:  1.6333341244323414
p2p_width:  0.26666679582568836
p2p_width:  1.6666674739105525
p2p_width:  0.30000014530389946
p2p_width:  1.7333341728669742
p2p_width:  0.6000002906077988
p2p_width:  -0.13333339791284407
Can't

#### Separate the units by layer and type:

In [12]:
layer_borders

[0, 100, 310, 430, 650, 850]

In [13]:
inh_units_layers_all_sessions = [[]]*nsessions; exc_units_layers_all_sessions = [[]]*nsessions;

ninh_nrns = 0; nexc_nrns = 0 # just counters used for checks
for k, session in enumerate(list_sessions):
    
    inh_units_layers_temp = [[]]*nlayers
    exc_units_layers_temp = [[]]*nlayers
    for i in range(1, len(layer_borders)):
        # mask to get units in between borders of a layer
        mask = np.logical_and(units_VISp_session_depth[k] <= layer_borders[i], units_VISp_session_depth[k] > layer_borders[i-1])

        temp_units_VISp_session_layers = units_VISp_session_depth[k][mask]
        #units_VISp_session_layers.append(temp_units_VISp_session_layers)
        
        # All units in layer gathered (not really used for anything)
        nrns_in_layer = temp_units_VISp_session_layers.keys().to_numpy()
        #print('nrns_in_layer: ', nrns_in_layer)
        
        # Separate inhibitory and excitatory units based on identification above
        temp_inh_units_layers = []
        temp_exc_units_layers = []
        j = 0
        while j < len(nrns_in_layer):
            if nrns_in_layer[j] in inh_nrns_list[k]:
                temp_inh_units_layers.append(nrns_in_layer[j])
                ninh_nrns += 1
            else:
                if layer_borders[i] != 100: # not supposed to be any excitatory neurons in layer 1
                    temp_exc_units_layers.append(nrns_in_layer[j])
                    nexc_nrns += 1
            j += 1
        inh_units_layers_temp[i-1] = temp_inh_units_layers
        exc_units_layers_temp[i-1] = temp_exc_units_layers
        print('inh_units_layers[i-1]: ', inh_units_layers_temp[i-1])
        print('exc_units_layers[i-1]: ', exc_units_layers_temp[i-1])
        print(layer_borders[i], (len(inh_units_layers_temp[i-1])+len(exc_units_layers_temp[i-1])))
        print(layer_borders[i], len(nrns_in_layer))
    
    inh_units_layers_all_sessions[k] = inh_units_layers_temp
    exc_units_layers_all_sessions[k] = exc_units_layers_temp
    
    

inh_units_layers[i-1]:  []
exc_units_layers[i-1]:  []
100 0
100 0
inh_units_layers[i-1]:  []
exc_units_layers[i-1]:  [950932445, 950932563, 950932578, 950932696, 950933960]
310 5
310 5
inh_units_layers[i-1]:  []
exc_units_layers[i-1]:  [950933924]
430 1
430 1
inh_units_layers[i-1]:  [950931254, 950931315, 950931423, 950933840]
exc_units_layers[i-1]:  [950931164, 950931181, 950931236, 950931272, 950931458, 950931363, 950931517, 950931533, 950931565, 950931581, 950931617, 950931805, 950931656, 950931727, 950931751, 950931770, 950931853, 950931878, 950931899, 950931959, 950932032, 950933907, 950932087, 950932102, 950933890]
650 29
650 29
inh_units_layers[i-1]:  [950930237, 950930407, 950930964, 950930985, 950933732]
exc_units_layers[i-1]:  [950930105, 950930145, 950930215, 950933698, 950930276, 950930340, 950930358, 950930375, 950930392, 950930423, 950930437, 950930454, 950930522, 950930795, 950930866, 950930888, 950934181, 950931004, 950931118, 950931043]
850 25
850 25


In [14]:
time_bin_start = -0.1 # seconds before stimulus onset
time_bin_end = 0.5 # seconds after stimulus onset
nbins_pr_second = 1000 # 1 ms bins
nbins = int((time_bin_end-time_bin_start)*nbins_pr_second+1)
time_bin_edges = np.linspace(time_bin_start, time_bin_end, nbins)

ratio_bins_second = (time_bin_end-time_bin_start)/nbins
print(ratio_bins_second)

0.0009983361064891847


In [15]:
start_twoi = 0 # start time window of interest (ms)
end_twoi = 100 # end time window of interest (ms)

# Index time window of interest for simulations
idx_start_twoi_sim = start_twoi+500 
idx_end_twoi_sim = end_twoi+500

# Index time window of interest for experimental data
idx_start_twoi_exp = int(start_twoi/(ratio_bins_second*1E3) - time_bin_edges[0]/ratio_bins_second)
idx_end_twoi_exp = int(end_twoi/(ratio_bins_second*1E3) - time_bin_edges[0]/ratio_bins_second)
len_idx_twoi_exp = idx_end_twoi_exp - idx_start_twoi_exp

In [16]:
color = 'white'
if color == 'white':
    stim_type = 0
    color_mask = 1
if color == 'black':
    stim_type = 1
    color_mask = -1

In [17]:
layerpop_names

['e23', 'e4', 'e5', 'e6', 'i1', 'i23', 'i4', 'i5', 'i6']

## Extract spike times and compute firing rates
- Note: This takes a long time. If you want to test things I'd recommend you to start with only first 10 sessions or so


In [18]:
# Firing rates of units in different layer populations separated by individual sessions 
fir_rates_all_cells_sessions = {}

for sesNr, session in enumerate(list_sessions):
    print('------------------------------------------------\n\n')
    print('Session: ', sesNr, ', Session label: ', list_session_ids[sesNr])
    
    # look at responses to the flash stimulus
    flash_250_ms_stimulus_presentation = session.stimulus_presentations[
        session.stimulus_presentations['stimulus_name'] == 'flashes'
    ]
    flash_250_ms_stimulus_presentation_ids = flash_250_ms_stimulus_presentation[
        session.stimulus_presentations.color == color_mask].index.values
    
    print('The stimulus was presented ', np.any(flash_250_ms_stimulus_presentation_ids))
    
    if np.any(flash_250_ms_stimulus_presentation_ids):
        fir_rate_all_cells_layers = {}
        for layPopNr, layPopName in enumerate(layerpop_names):
            print('Pop: ', layPopName)
            if layPopName[0] == 'i':
                units_of_interest = inh_units_layers_all_sessions[sesNr][layPopNr-4]
                n_units_in_pop = len(units_of_interest)
                print('n_inh_units_in_layer: ', n_units_in_pop)
            else:
                units_of_interest = exc_units_layers_all_sessions[sesNr][layPopNr+1]
                n_units_in_pop = len(units_of_interest)
                print('n_exc_units_in_pop: ', n_units_in_pop)
            
            if n_units_in_pop == 0:
                fir_rate_all_cells_layers[layPopName] = []
            else:
                temp_spike_counts = session.presentationwise_spike_counts(
                bin_edges=time_bin_edges,
                stimulus_presentation_ids=flash_250_ms_stimulus_presentation_ids,
                unit_ids=units_of_interest
                )
                
                # Transpose because my personal preference is that the unit is the first dimension
                # (time is second dimension, trials is third dimension)
                print('shape temp_spike_counts.T: ', np.shape(temp_spike_counts.T))
                
                # Compute average and scale to Hz
                temp_trial_avg = np.mean(temp_spike_counts.T, axis = 2)/ratio_bins_second
                
                fir_rate_all_cells_layers[layPopName] = temp_trial_avg
                
        fir_rates_all_cells_sessions[list_session_ids_string[sesNr]] = fir_rate_all_cells_layers

------------------------------------------------


Session:  0 , Session label:  715093703


/Users/atleeskelandrimehaug/opt/anaconda3/envs/gen_env/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]
/Users/atleeskelandrimehaug/opt/anaconda3/envs/gen_env/lib/python3.7/site-packages/allensdk/brain_observatory/ecephys/ecephys_session.py:1093: UserWarning: Session includes invalid time intervals that could be accessed with the attribute 'invalid_times',Spikes within these intervals are invalid and may need to be excluded from the analysis.
  warnings.warn("Session includes invalid time intervals that could be accessed with the attribute 'invalid_times',"


The stimulus was presented  True
Pop:  e23
n_exc_units_in_pop:  5
shape temp_spike_counts.T:  (5, 600, 75)
Pop:  e4
n_exc_units_in_pop:  1
shape temp_spike_counts.T:  (1, 600, 75)
Pop:  e5
n_exc_units_in_pop:  25
shape temp_spike_counts.T:  (25, 600, 75)
Pop:  e6
n_exc_units_in_pop:  20
shape temp_spike_counts.T:  (20, 600, 75)
Pop:  i1
n_inh_units_in_layer:  0
Pop:  i23
n_inh_units_in_layer:  0
Pop:  i4
n_inh_units_in_layer:  0
Pop:  i5
n_inh_units_in_layer:  4
shape temp_spike_counts.T:  (4, 600, 75)
Pop:  i6
n_inh_units_in_layer:  5
shape temp_spike_counts.T:  (5, 600, 75)
